In [15]:
import pandas as pd
import inro.modeller as m

mm = m.Modeller()
eb = mm.emmebank
sc_1 = eb.scenario(6002)
sc_2 = eb.scenario(4310)
ntwk1 = sc_1.get_network()
ntwk2 = sc_2.get_network()

In [16]:
ol_nodes = []
for link in ntwk2.links():
    if link["@sp_link"] == 1:
        if link.i_node not in ol_nodes:
            ol_nodes.append(link.i_node)
        if link.j_node not in ol_nodes:
            ol_nodes.append(link.j_node)

In [4]:
ol_nodes

[Node(97093),
 Node(98022),
 Node(98087),
 Node(98085),
 Node(98148),
 Node(98088),
 Node(98092),
 Node(98093),
 Node(98095),
 Node(98096),
 Node(98097),
 Node(98098),
 Node(98099),
 Node(98100),
 Node(98147)]

In [5]:
for node in ol_nodes:
    if ntwk1.node(node.id) is None:
        newNode = ntwk1.create_node(node.id, is_centroid = False)
        newNode.x = node.x
        newNode.y = node.y

In [6]:
sc_1.publish_network(ntwk1)

In [21]:
ntwk1.delete_link(97093,98022)
ntwk1.delete_link(98022,97093)

In [22]:
for link in ntwk2.links():
    if link["@sp_link"] == 1:
        if ntwk1.link(link.i_node,link.j_node) is None:
            olLink = ntwk1.create_link(link.i_node.id,link.j_node.id,link.modes)
            newVerts = link.vertices
            olLink.vertices = newVerts
            olLink.length = link.length
            for att in linkatts_1:
                olLink[att] = link[att]  

In [25]:
sc_1.publish_network(ntwk1)

In [24]:
for node in ol_nodes:
    for link in node.incoming_links():
        if ntwk1.link(link.i_node,link.j_node) is None:
            new_link = ntwk1.create_link(link.i_node.id,link.j_node.id,link.modes)
            for att in linkatts_1:
                new_link[att] = link[att]        
    for link in node.outgoing_links():
        if ntwk1.link(link.i_node,link.j_node) is None:
            new_link = ntwk1.create_link(link.i_node.id,link.j_node.id,link.modes)
            for att in linkatts_1:
                new_link[att] = link[att]

In [7]:
linkatts_1 = ntwk1.attributes("LINK")

In [8]:
linkatts_2 = ntwk2.attributes("LINK")

In [18]:
linkatts_1.remove(u'@ol_link')

In [12]:
linkatts_2

[u'@bike_speed',
 u'@bike_time',
 u'@bln',
 u'@flag_l',
 u'@gtha_links',
 u'@hov2_km',
 u'@hov3_km',
 u'@idpro',
 u'@is_hov2',
 u'@is_hov3',
 u'@sp_link',
 u'@tag1',
 u'@tag2',
 u'@tag3',
 u'@toll1',
 u'@toll2',
 u'@toll3',
 u'@toll_htruck',
 u'@toll_km',
 u'@toll_mtruck',
 u'@toll_zone',
 u'@volau1_nt',
 u'@volau1_toll',
 u'@volau2_nt_hov',
 u'@volau2_nt_nh',
 u'@volau2_toll_hov',
 u'@volau2_toll_nh',
 u'@volau3_nt_hov',
 u'@volau3_nt_nh',
 u'@volau3_toll_hov',
 u'@volau3_toll_nh',
 u'@volau_htruck',
 u'@volau_ltruck',
 u'@volau_mtruck',
 u'@walk_speed',
 u'@walk_time',
 u'data1',
 u'data2',
 u'data3',
 u'length',
 u'num_lanes',
 u'type',
 u'volume_delay_func']

In [12]:
linkatts_2.remove(u'additional_volume')
linkatts_2.remove(u'auto_time')
linkatts_2.remove(u'auto_volume')
linkatts_2.remove(u'aux_transit_volume')

In [14]:
for zone in ntwk1.centroids():
    if zone["@ol_zone"] == 1:
        
        zone2 = ntwk2.node(zone.id)
        zone.x = zone2.x
        zone.y = zone2.y
        
        for link in zone.incoming_links():
            ntwk1.delete_link(link.i_node.id,link.j_node.id)
        for link in zone.outgoing_links():
            ntwk1.delete_link(link.i_node.id,link.j_node.id)    
        
        for link in zone2.incoming_links():
            new_link = ntwk1.create_link(link.i_node.id,link.j_node.id,link.modes)
            for att in linkatts_2:
                new_link[att] = link[att]        
        for link in zone2.outgoing_links():
            new_link = ntwk1.create_link(link.i_node.id,link.j_node.id,link.modes)
            for att in linkatts_2:
                new_link[att] = link[att]

In [15]:
sc_1.publish_network(ntwk1)

In [19]:
ntwk1.link(10611,97085).modes == set([ntwk1.mode('t')])

True

In [21]:
for node in ol_nodes:
    for link in node.incoming_links():
        if link.modes == set([ntwk1.mode('t')]):
            ntwk1.delete_link(link.i_node.id,link.j_node.id)
    for link in node.outgoing_links():
        if link.modes == set([ntwk1.mode('t')]):
            ntwk1.delete_link(link.i_node.id,link.j_node.id)
    newNode = ntwk2.node(node.id)
    for link in newNode.incoming_links():
        if link.modes == set([ntwk2.mode('t')]):
            new_link = ntwk1.create_link(link.i_node.id,link.j_node.id,link.modes)
            for att in linkatts_2:
                new_link[att] = link[att] 
    for link in newNode.outgoing_links():
        if link.modes == set([ntwk2.mode('t')]):
            new_link = ntwk1.create_link(link.i_node.id,link.j_node.id,link.modes)
            for att in linkatts_2:
                new_link[att] = link[att]

In [22]:
sc_1.publish_network(ntwk1)